# Karaka PN methylation basecalling.
## Data & workflow
- R10 chemistry
- Sequenced on a minion in house.
- from `final_summary_PAW31230_f467ca5d_9b2e8d74.txt`: protocol=sequencing/sequencing_MIN114_DNA_e8_2_400K:FLO-MIN114:SQK-LSK114:400

In [ ]:
DORADO=/workspace/hraijc/git_clones/dorado-1.0.2-linux-x64/bin/dorado
WKDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/
READDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/Karaka_PN/Karaka_PN/20260120_1200_MN19482_FBC78246_293559bf/pod5
mkdir -p $WKDIR/log

# 00.Sup basecalling with 5mC. Skipping 6mA mods due to low quality calls

## Karaka Run 1

In [ ]:
DORADO=/workspace/hraijc/git_clones/dorado-1.0.2-linux-x64/bin/dorado
WKDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/
READDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/Karaka_PN/Karaka_PN/20260120_1200_MN19482_FBC78246_293559bf/pod5
mkdir -p $WKDIR/log

LOG=$WKDIR/log
OUT=$WKDIR/basecalling
mkdir -p ${OUT}
cd ${OUT}

sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=160G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=47:00:00
#SBATCH --gres=gpu:1

cd $OUT

$DORADO download --model dna_r10.4.1_e8.2_400bps_sup@v5.0.0
$DORADO download --model dna_r10.4.1_e8.2_400bps_sup@v5.0.0_5mC_5hmC@v1

$DORADO basecaller dna_r10.4.1_e8.2_400bps_sup@v5.0.0 \
    $READDIR \
     --modified-bases-models dna_r10.4.1_e8.2_400bps_sup@v5.0.0_5mC_5hmC@v1 \
    --min-qscore 15 > KarakaPN_ONT.bam


EOF


In [ ]:
DORADO=/workspace/hraijc/git_clones/dorado-1.0.2-linux-x64/bin/dorado
WKDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/
READDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/Karaka_PN_2/Karaka_PN_2/20260121_0937_MN19482_FBC78246_07078daa/pod5
mkdir -p $WKDIR/log

LOG=$WKDIR/log
OUT=$WKDIR/basecalling
mkdir -p ${OUT}
cd ${OUT}

sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=160G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=47:00:00
#SBATCH --gres=gpu:1

cd $OUT
$DORADO basecaller dna_r10.4.1_e8.2_400bps_sup@v5.0.0 \
    $READDIR \
     --modified-bases-models dna_r10.4.1_e8.2_400bps_sup@v5.0.0_5mC_5hmC@v1 \
    --min-qscore 15 > KarakaPN_ONT2.bam



EOF


## bam to fastq and qc

In [ ]:
WKDIR=/workspace/hraijc/TEMP/KarakaPN_ONT/basecalling/
LOG=$WKDIR/log
mkdir -p $LOG
cd $WKDIR


sbatch << EOF
#!/bin/bash
#SBATCH -J nanocomp
#SBATCH -o $LOG/%J.out 
#SBATCH -e $LOG/%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem=140G
#SBATCH --time=14:00:00

cd $WKDIR

# load packages
module load samtools/1.20
module load htslib/1.20

samtools view -@ 2 -O fastq KarakaPN_ONT.bam | bgzip -@ 2 > KarakaPN_ONT1.fastq.gz
samtools view -@ 2 -O fastq KarakaPN_ONT2.bam | bgzip -@ 2 > KarakaPN_ONT2.fastq.gz
cat KarakaPN_ONT1.fastq.gz KarakaPN_ONT2.fastq.gz > KarakaPN_ONT.fastq.gz

module unload samtools/1.20
module unload htslib/1.20

# QC
module load conda
conda deactivate

module load nanopack

NanoPlot --fastq KarakaPN_ONT.fastq.gz --outdir nanoplot_out


EOF

#check outputs and delete scratch files. 
rm -rf KarakaPN_ONT1.fastq.gz
rm -rf KarakaPN_ONT2.fastq.gz

## Move to input

In [ ]:
mv /powerplant/workspace/hraijc/TEMP/KarakaPN_ONT/ /powerplant/workspace/hraijc/TEMP/KarakaPN_ONT_MinION_2026
mv /powerplant/workspace/hraijc/TEMP/KarakaPN_ONT_MinION_2026 /powerplant/input/taonga/genomic/plant/Corynocarpus/laevigatus

export odir=/input/taonga/genomic/plant/Corynocarpus/laevigatus/KarakaPN_ONT_MinION_2026
find $odir -type d -exec chmod 755 {} \; 2>/dev/null
find $odir -type f -exec chmod 444 {} \; 2>/dev/null

